In [1]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author):
    pattern = '%3s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author))

In [20]:
# coding=utf-8
import requests
from requests_html import HTML
def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text,
    }

    try:
        # 正常狀況取得資料
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']
    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    return meta
search_endpoint_url = 'https://www.ptt.cc/bbs/movie/search'
resp = requests.get(search_endpoint_url, params={'q': '鋒迴路轉', 'page' : 4})

post_entries = parse_article_entries(resp.text)  # [沿用]
metadata = [parse_article_meta(entry) for entry in post_entries]  # [沿用]

for entry in post_entries:
    meta = parse_article_meta(entry)
#     print(meta)  # result of setp-3

    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  # for below results

  7	[請益] 鋒迴路轉問題，有雷慎入                     11/30	jim2012
 16	[選片] 賽道狂人 鋒迴路轉 選片～～                 11/30	x5533007
  9	[選片] 瓶中信 or 鋒迴路轉                         11/30	KKKKKvs333
 14	[選片] 鋒迴路轉/82年的金智英/賽道狂人             11/30	f40075566
 22	[新聞] 《鋒迴路轉》全美票房超預期暴走 導演爆      11/30	LAMessi
  3	[爽雷] 鋒迴路轉，好的推理故事不用燒腦             11/30	linhsiuwei
  1	[好雷]《鋒迴路轉》推理的好才是王道                11/30	allshine
 19	[選片] 鋒迴路轉/布魯克林孤兒/夕霧花園             11/30	jason748
 14	[請益] 鋒迴路轉問題，極有雷慎入                   11/30	creepygirl
 20	[新聞] 導演想把《鋒迴路轉》拍成系列電影           11/29	hahaha0204
 26	[好雷] 鋒迴路轉                                   11/29	kellykelly
 30	[選片] 賽道狂人/鋒迴路轉 選片請益                 11/29	jerry9233
 12	[好雷] 《鋒迴路轉》無雷感想 (推文有雷)            11/29	AquaX
 13	[好無雷] 鋒迴路轉 意外的簡單有趣                  11/29	kyo2003
 10	[好雷] 鋒迴路轉 出乎意料好看                      11/28	WHeisenberg
  2	[好雷] 鋒迴路轉-八點檔式的推理小說                11/27	blueskymaple
  8	[好雷] <鋒迴路轉> 真的峰迴路轉                    11/27	sandy7521
 17	[新聞] 克里斯伊凡《鋒迴路轉》燒腦爆好評 還沒上映網已敲碗：快拍10/26	kenny130017

In [ ]:
from multiprocessing import Pool